    回归树

CART算法：

每次对数据集进行拆分时都进行二分

叶子结点的值是均值

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
iris = pd.read_csv('/Users/nanyueming/Documents/学习/python/机器学习与量化投资/机器学习模型/iris.data' , 
                   header = None , names = ['x1','x2','x3','x4','type'])
iris.head()

,x1,x2,x3,x4,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
flowers = list(set(iris.type))

In [4]:
iris.type = pd.Categorical(iris.type).codes

In [5]:
iris[iris['x1'] > 4].drop('x1',axis = 1).head()

,x2,x3,x4,type
0,3.5,1.4,0.2,0
1,3.0,1.4,0.2,0
2,3.2,1.3,0.2,0
3,3.1,1.5,0.2,0
4,3.6,1.4,0.2,0


In [6]:
def split(data , feature , value):
    
    #根据所给的特征和值分割数据集
    
    left_data = data[data[feature] > value].drop(feature , axis = 1)
    right_data = data[data[feature] <= value].drop(feature , axis = 1)
    
    return left_data , right_data

In [7]:
def sqr_error(data):
    
    #损失函数：平方误差
    
    return np.var(data.iloc[:,-1]) * len(data.iloc[:,-1])

In [8]:
def best_cut(data):
    
    #寻找最佳的分割特征和值
    
    best_Sl = np.inf
    best_Sr = np.inf
    best_S = np.inf
    s = 0.0
    bestFeature = None
    bestValue = 0.0
    
    for x_feature in data.columns[:-1]:
        for value in list(set(data[x_feature])):
            left_data , right_data = split(data , x_feature , value)
            sl = sqr_error(left_data)
            sr = sqr_error(right_data)
            s = sl + sr
            
            if sl <= best_Sl and sr <= best_Sr and s < best_S:
                bestFeature = x_feature
                bestValue = value
                best_Sl = sl
                best_Sr = sr
                best_S = s
                
    
    return bestFeature , bestValue

In [9]:
def CART_Tree(data):
    
    #种树
    
    #如果所有特征都一样，停止分割
    if len(set(data.iloc[:,-1])) == 1:
        return flowers[data.iloc[:,-1].mode()[0]]
    
    #如果遍历完所有特征了，停止分割
    if len(data.columns) == 1:
        return flowers[data.iloc[:,-1].mode()[0]]
    
    feature , value = best_cut(data)
    

    cartTree = {}
    left_tree , right_tree = split(data , feature , value)
    
    cartTree[feature + '>' + str(value)] = CART_Tree(left_tree)
    cartTree[feature + '<' + str(value)] = CART_Tree(right_tree)
    

    return cartTree 

In [10]:
CART_Tree(iris)

{'x3>1.9': {'x4>1.3': {'x1>4.9': {'x2>2.8': 'Iris-virginica',
    'x2<2.8': 'Iris-virginica'},
   'x1<4.9': 'Iris-virginica'},
  'x4<1.3': 'Iris-versicolor'},
 'x3<1.9': 'Iris-setosa'}